In [1]:
import subprocess as sp
import os
import pandas as pd

In [2]:
if not os.path.exists("vicuna-7B-GPTQ-4bit-128g"):
    !git clone https://huggingface.co/TheBloke/vicuna-7B-GPTQ-4bit-128g

In [3]:
!pip uninstall --yes alpaca_lora_4bit
!pip install git+https://github.com/johnsmith0031/alpaca_lora_4bit@65e5495ce5ffcd669408ad5d3e5d6e0f71739e9f

Found existing installation: alpaca-lora-4bit 0.1.2
Uninstalling alpaca-lora-4bit-0.1.2:
  Successfully uninstalled alpaca-lora-4bit-0.1.2
  Cloning https://github.com/johnsmith0031/alpaca_lora_4bit (to revision 65e5495ce5ffcd669408ad5d3e5d6e0f71739e9f) to c:\users\alex4321\appdata\local\temp\pip-req-build-j7u0l1ou
  Resolved https://github.com/johnsmith0031/alpaca_lora_4bit to commit 65e5495ce5ffcd669408ad5d3e5d6e0f71739e9f
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached peft-0.3.0.dev0-py3-none-any.whl
  Cloning https://github.com/huggingface/transformers.git to c:\users\alex4321\appdata\local\temp\pip-install-ozp91o6y\transformers_3e4984fe29cf470cad6100c0f57a644d
  Resolved https://github.com/huggingface/transformers.git to commit 1e9da2b0a6ef964c2cf72dd715dbee991a3f49fa
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: start

  Running command git clone --filter=blob:none --quiet https://github.com/johnsmith0031/alpaca_lora_4bit 'C:\Users\alex4321\AppData\Local\Temp\pip-req-build-j7u0l1ou'
  Running command git rev-parse -q --verify 'sha^65e5495ce5ffcd669408ad5d3e5d6e0f71739e9f'
  Running command git fetch -q https://github.com/johnsmith0031/alpaca_lora_4bit 65e5495ce5ffcd669408ad5d3e5d6e0f71739e9f
  Running command git checkout -q 65e5495ce5ffcd669408ad5d3e5d6e0f71739e9f
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git 'C:\Users\alex4321\AppData\Local\Temp\pip-install-ozp91o6y\transformers_3e4984fe29cf470cad6100c0f57a644d'


In [4]:
code = """
from alpaca_lora_4bit.autograd_4bit import load_llama_model_4bit_low_ram, Autograd4bitQuantLinear, switch_backend_to

switch_backend_to("cuda")

model, tokenizer = load_llama_model_4bit_low_ram("vicuna-7B-GPTQ-4bit-128g",
                                                 "vicuna-7B-GPTQ-4bit-128g/vicuna-7B-GPTQ-4bit-128g.safetensors",
                                                 groupsize=128)

model.half();
for n, m in model.named_modules():
    if isinstance(m, Autograd4bitQuantLinear):
        if m.is_v1_model:
            m.zeros = m.zeros.half()
        m.scales = m.scales.half()
        m.bias = m.bias.half()

modules = list(model.modules())
modules = [m for m in modules if isinstance(m, Autograd4bitQuantLinear)]
for i, m in enumerate(modules):
    assert not torch.isnan(m.bias).any().item()
    assert not torch.isinf(m.bias).any().item()
"""
with open("__test.py", "w") as dst:
    dst.write(code)

In [5]:
exit_codes = []
for i in range(10):
    exit_code = sp.Popen(["python", "__test.py"]).wait()
    exit_codes.append(exit_code)

pd.Series(exit_codes).value_counts()

1    10
Name: count, dtype: int64